# Machine Learning Engineer Nanodegree
## Deep Learning
## Project: Build a Digit Recognition Program

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

----
## Step 1: Design and Test a Model Architecture
Design and implement a deep learning model that learns to recognize sequences of digits. Train the model using synthetic data generated by concatenating character images from [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) or [MNIST](http://yann.lecun.com/exdb/mnist/). To produce a synthetic sequence of digits for testing, you can for example limit yourself to sequences up to five digits, and use five classifiers on top of your deep network. You would have to incorporate an additional ‘blank’ character to account for shorter number sequences.

There are various aspects to consider when thinking about this problem:
- Your model can be derived from a deep neural net or a convolutional network.
- You could experiment sharing or not the weights between the softmax classifiers.
- You can also use a recurrent network in your deep neural net to replace the classification layers and directly emit the sequence of digits one-at-a-time.

You can use ** Keras ** to implement your model. Read more at [keras.io](https://keras.io/).

Here is an example of a [published baseline model on this problem](http://static.googleusercontent.com/media/research.google.com/en//pubs/archive/42241.pdf). ([video](https://www.youtube.com/watch?v=vGPI_JvLoN0)). You are not expected to model your architecture precisely using this model nor get the same performance levels, but this is more to show an exampe of an approach used to solve this particular problem. We encourage you to try out different architectures for yourself and see what works best for you. Here is a useful [forum post](https://discussions.udacity.com/t/goodfellow-et-al-2013-architecture/202363) discussing the architecture as described in the paper and here is [another one](https://discussions.udacity.com/t/what-loss-function-to-use-for-multi-digit-svhn-training/176897) discussing the loss function.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

## Get the data for notMNIST

In [1]:
from urllib.request import urlretrieve
import tarfile
from os.path import isdir
from os import remove

def folder_file_name(urlpath):
    """
    Takes a URL and returns the characters after the final '/' as
    the filename. In the filename, everything up until the first 
    period is declared to be the foldername, i.e. the folder-name
    to which a tar file would be unpackaged.
    """
    # We first find the index where the name of the file begins
    indexname = urlpath.rfind("/")
    # Now we get the file name
    filename = urlpath[indexname+1:]
    # We will do the same for the folder name
    folderindex = filename.find(".")
    foldername = filename[:folderindex]
    return filename, foldername

def download_and_unpackage(urlpath):
    """
    Downloads a file given by the url address and unpackages it
    into the same directory. It then removes the compressed file.
    """
    filename, foldername = folder_file_name(urlpath)
    # We only want to download and unpackage if we haven't already done it
    if isdir(foldername) != True:
        urlretrieve(urlpath, filename)
        with tarfile.open(filename, mode='r:gz') as compressed_file:
            compressed_file.extractall()
            compressed_file.close()
        remove(filename)

# Now the data for notMNIST
download_and_unpackage("http://commondatastorage.googleapis.com/books1000/notMNIST_small.tar.gz")
download_and_unpackage("http://commondatastorage.googleapis.com/books1000/notMNIST_large.tar.gz")

## Preprocess data

In [2]:
import scipy.ndimage
import os
import numpy as np
from sklearn.preprocessing import LabelBinarizer

import matplotlib.pyplot as plt
%matplotlib inline

### Find path names of files and prepare one-hot encoder

In [47]:
# We first get all path-names for the training and testing images
training_pathnames = [[folderandfiles[0]+"/"+imname for imname in folderandfiles[2]] 
                          for folderandfiles in os.walk("./notMNIST_large") if folderandfiles[2]!=[]]
testing_pathnames = [[folderandfiles[0]+"/"+imname for imname in folderandfiles[2]] 
                          for folderandfiles in os.walk("./notMNIST_small") if folderandfiles[2]!=[]]
# training_pathnames has the structure:
#[[list of path-names in first folder], [list of paths in second folder], ...]

def get_letter(filepath):
    """
    Returns the letter corresponding to an image found in filepath
    """
    # The letter is given by the name of the folder in which we find the image
    indexname = filepath.rfind("/")
    letter = filepath[indexname-1:indexname]
    return letter

# In each folder all images depict the same letter
all_letters = np.sort([get_letter(pathname[0]) for pathname in training_pathnames])

# We may now make the function that one-hot-encodes letters into arrays
enc = LabelBinarizer()
enc.fit(all_letters)

def one_hot_encode(list_of_letters):
    """
    One hot encode a list of letters. Returns a one-hot encoded vector for each letter.
    """
    return enc.transform(list_of_letters)

# We now flatten the lists of path names
training_pathnames = np.array(sum(training_pathnames, []))
testing_pathnames = np.array(sum(testing_pathnames, []))

### Load images and labels into arrays

We first normalize the pixel-values to lie between 0 and 1. We also reshape each image to be a 3-dimensional array: (x_length, y_length, color_channels)

In [48]:
def load_normalize_image(path):
    """
    Takes the directory path of an image and returns a normalized
    3-dimensional array representing that image.
    """
    # First we load the image
    try:
        imagearray = scipy.ndimage.imread(path)
        # Now we normalize it 
        imagearray = imagearray / 255
        # We reshape it to be 3-dimensional: x-dim, y-dim, num_colors
        imagearray = imagearray.reshape(imagearray.shape + (1,))
        return imagearray
    except:
        # Some images are broken in the database; these will raise errors.
        pass
    
def array_all_images(list_of_path_names):
    """
    Takes a list of directory paths to images and returns a 4-dimensional array
    containing the pixel-data of those images. The shape is:
    (num_images, x_dim, y_dim, num_colors)
    """
    all_images = [load_normalize_image(path) for path in list_of_path_names]
    # Some of these might be None since the function load_normalize_image
    # does not load broken images. We now remove these Nones.
    all_images = np.array(list(filter(None.__ne__, all_images)))
    return all_images

def array_all_labels(list_of_path_names):
    """
    Takes a list of directory paths to images and returns a 2-dimensional array
    containing the one-hot-encoded labels of those images
    """
    all_labels = one_hot_encode([get_letter(path) for path in list_of_path_names])
    return all_labels

#training_images = array_all_images(training_pathnames)
#training_labels = array_all_labels(training_pathnames)
testing_images = array_all_images(testing_pathnames)
testing_labels = array_all_labels(testing_pathnames)

In [49]:
# Need to turn trainig into batches!

In [37]:
dalist = range(len(training_pathnames))
batch_size = 2**10

In [41]:
batch_indices = [dalist[ii: ii+batch_size] for ii in range(0, len(dalist), batch_size)]

In [46]:
np.array(training_pathnames)[batch_indices[0]]

array(['./notMNIST_large/E/Q29ycG9yYXRlU0V4cGVydEJRLUJvbGQub3Rm.png',
       './notMNIST_large/E/SWdyZW5vYmxlLUhlYXZ5Lm90Zg==.png',
       './notMNIST_large/E/VHJpbml0ZU5vMi1JdGFsaWMub3Rm.png', ...,
       './notMNIST_large/E/Q2VyaWdvU3RkLUJvb2sub3Rm.png',
       './notMNIST_large/E/QWx0ZW11c1NwaXJhbHMub3Rm.png',
       './notMNIST_large/E/TWF0cml4SW5saW5lRXh0cmFCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png'], 
      dtype='<U111')

### Display an image

In [4]:
def display_image(imagearray):
    array_to_plot = imagearray.reshape((imagearray.shape[0], imagearray.shape[1]))
    print("Image shape: {}".format(imagearray.shape))
    plt.imshow(array_to_plot, cmap="gray")
    plt.axis("off")
    plt.show()
    
#display_image()

### Question 1
_What approach did you take in coming up with a solution to this problem?_

**Answer:** 

### Question 2
_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)_

**Answer:**

### Question 3
_How did you train your model? How did you generate your synthetic dataset?_ Include examples of images from the synthetic data you constructed.

**Answer:**

----
## Step 2: Train a Model on a Realistic Dataset
Once you have settled on a good architecture, you can train your model on real data. In particular, the [Street View House Numbers (SVHN)](http://ufldl.stanford.edu/housenumbers/) dataset is a good large-scale dataset collected from house numbers in Google Street View. Training on this more challenging dataset, where the digits are not neatly lined-up and have various skews, fonts and colors, likely means you have to do some hyperparameter exploration to perform well.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

### Question 4
_Describe how you set up the training and testing data for your model. How does the model perform on a realistic dataset?_

**Answer:**

### Question 5
_What changes did you have to make, if any, to achieve "good" results? Were there any options you explored that made the results worse?_

**Answer:**

### Question 6
_What were your initial and final results with testing on a realistic dataset? Do you believe your model is doing a good enough job at classifying numbers correctly?_

**Answer:**

----
## Step 3: Test a Model on Newly-Captured Images

Take several pictures of numbers that you find around you (at least five), and run them through your classifier on your computer to produce example results. Alternatively (optionally), you can try using OpenCV / SimpleCV / Pygame to capture live images from a webcam and run those through your classifier.

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 7
_Choose five candidate images of numbers you took from around you and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult?_

**Answer:**

### Question 8
_Is your model able to perform equally well on captured pictures or a live camera stream when compared to testing on the realistic dataset?_

**Answer:**

### Optional: Question 9
_If necessary, provide documentation for how an interface was built for your model to load and classify newly-acquired images._

**Answer:** Leave blank if you did not complete this part.

----
### Step 4: Explore an Improvement for a Model

There are many things you can do once you have the basic classifier in place. One example would be to also localize where the numbers are on the image. The SVHN dataset provides bounding boxes that you can tune to train a localizer. Train a regression loss to the coordinates of the bounding box, and then test it. 

### Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your code implementation goes here.
### Feel free to use as many code cells as needed.



### Question 10
_How well does your model localize numbers on the testing set from the realistic dataset? Do your classification results change at all with localization included?_

**Answer:**

### Question 11
_Test the localization function on the images you captured in **Step 3**. Does the model accurately calculate a bounding box for the numbers in the images you found? If you did not use a graphical interface, you may need to investigate the bounding boxes by hand._ Provide an example of the localization created on a captured image.

**Answer:**

----
## Optional Step 5: Build an Application or Program for a Model
Take your project one step further. If you're interested, look to build an Android application or even a more robust Python program that can interface with input images and display the classified numbers and even the bounding boxes. You can for example try to build an augmented reality app by overlaying your answer on the image like the [Word Lens](https://en.wikipedia.org/wiki/Word_Lens) app does.

Loading a TensorFlow model into a camera app on Android is demonstrated in the [TensorFlow Android demo app](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/android), which you can simply modify.

If you decide to explore this optional route, be sure to document your interface and implementation, along with significant results you find. You can see the additional rubric items that you could be evaluated on by [following this link](https://review.udacity.com/#!/rubrics/413/view).

### Optional Implementation
Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:


### Your optional code implementation goes here.
### Feel free to use as many code cells as needed.



### Documentation
Provide additional documentation sufficient for detailing the implementation of the Android application or Python program for visualizing the classification of numbers in images. It should be clear how the program or application works. Demonstrations should be provided. 

_Write your documentation here._

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.